<a href="https://colab.research.google.com/github/chiangtinhung-rgb/Matcha/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***日常支出速算與分攤（作業一）***

目標：從 Sheet 讀「消費紀錄」→ 計總額/分類小計/AA分攤 → 寫回 Sheet Summary 分頁。

AI 點子：請模型總結本日消費並給予預算建議（例如「餐飲支出過多」）。

Sheet 欄位：date, time, item, amount, category, money, pay_way, payer_name, location, notes

GoogleSheet: https://docs.google.com/spreadsheets/d/1o4BY-VlRvKt0VFxY0pLuPaQKsLoRki8W9fDMaYgxGbI/edit?gid=0#gid=0

In [1]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
import datetime
from types import new_class
creds, _ = default()
gc = gspread.authorize(creds)

In [2]:
import pandas as pd
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1o4BY-VlRvKt0VFxY0pLuPaQKsLoRki8W9fDMaYgxGbI/edit?gid=0#gid=0')

In [ ]:


# 讓使用者輸入資料
date_str = input("請輸入日期 (格式：YYYY-MM-DD): ")
# 檢查日期格式是否正確


datetime.datetime.strptime(date_str, '%Y-%m-%d')

time_str = input("請輸入時間 (格式：HH:MM): ")
# 檢查時間格式是否正確
datetime.datetime.strptime(time_str, '%H:%M')

item = input("請輸入品項:")
amount = input("請輸入數量:")
category = input("請輸入類別 (例如：'餐飲', '交通', '娛樂', '生活用品', '學習', '收入', '其他'):")
money = float(input("請輸入金額:"))
pay_way = input("請輸入支付方式  (例如：'現金', '信用卡', '電子支付', '轉帳', '其他'):")
payer_name = input("請輸入付款人名字: ")
location = input("請輸入購買地點: ")
notes = input("請輸入備註/交易描述 (例如：朋友生日禮物、團購運費): ")

In [ ]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('工作表1').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()

In [ ]:
print(date_str, time_str, item, amount, category, money, pay_way, payer_name, location, notes)

In [ ]:
date_str

In [ ]:
time_str


In [ ]:
item

In [ ]:
amount

In [ ]:
category

In [ ]:
money

In [ ]:
pay_way

In [ ]:
payer_name

In [ ]:
location

In [ ]:
notes

In [ ]:
# 定義您希望 DataFrame 擁有的唯一欄位 (Column Index)
UNIQUE_COLUMNS = ['日期', '時間', '品項', '數量', '金額', '類別', '支付方式', '付款人', '地點', '備註']

# 創建一個空的 DataFrame，並強制使用唯一的欄位名稱
df = pd.DataFrame(columns=UNIQUE_COLUMNS)

# 創建包含新資料的 DataFrame
data = pd.DataFrame([{
    '日期': date_str,
    '時間': time_str,
    '品項': item,
    '數量': amount,
    '金額': money,
    '類別': category,
    '支付方式': pay_way,
    '付款人': payer_name,
    '地點': location,
    '備註': notes
}])

# 現在合併就不會有問題，因為 df 的欄位是唯一的
df = pd.concat([df, data], ignore_index=True)


In [ ]:
df

In [ ]:
# 將 DataFrame 轉換成列表的列表 (list of lists)，這是 gspread 支援的資料格式
data_to_write = df.values.tolist()

# 第一步：取得工作表物件
worksheet = gsheets.worksheet('工作表1')

# 第二步：將資料寫入工作表
worksheet.append_rows(values=data_to_write, value_input_option='USER_ENTERED')
print("資料已成功寫入 Google 工作表！")

In [ ]:
import gradio as gr
import datetime
import gspread
from google.colab import auth
from google.auth import default
import plotly.express as px

# 欄位列表與設定
SHEET_URL = "https://docs.google.com/spreadsheets/d/1o4BY-VlRvKt0VFxY0pLuPaQKsLoRki8W9fDMaYgxGbI/edit?gid=0#gid=0"
WORKSHEET_NAME = "工作表1"
REQUIRED_COLUMNS = ["日期", "時間", "分類", "品項", "金額", "付款人", "地點", "支付方式", "備註"]
_auth_done = False
_gc = None
_ws = None

In [ ]:
# --- 輔助函數 (維持不變) ---
def _ensure_auth_and_ws():
    global _auth_done, _gc, _ws
    if not _auth_done:
        auth.authenticate_user()
        creds, _ = default()
        _gc = gspread.authorize(creds)
        _auth_done = True
    if _ws is None:
        gs = _gc.open_by_url(SHEET_URL)
        _ws = gs.worksheet(WORKSHEET_NAME)
        _ensure_headers()
    return _ws

def _ensure_headers():
    rows = _ws.get_all_values()
    if not rows:
        _ws.append_row(REQUIRED_COLUMNS, value_input_option="USER_ENTERED")
        return
    header = rows[0]
    if header != REQUIRED_COLUMNS:
        existing = {h: idx for idx, h in enumerate(header)}
        _ws.update('1:1', [REQUIRED_COLUMNS])

        if len(rows) > 1:
            new_rows = []
            for r in rows[1:]:
                mapping = {col: (r[existing[col]] if col in existing and existing[col] < len(r) else "")
                           for col in REQUIRED_COLUMNS}
                new_rows.append([mapping[c] for c in REQUIRED_COLUMNS])
            _ws.resize(rows=1)
            _ws.append_rows(new_rows, value_input_option="USER_ENTERED")

def _read_df():
    ws = _ensure_auth_and_ws()
    values = ws.get_all_values()
    if not values:
        return pd.DataFrame(columns=REQUIRED_COLUMNS)
    df = pd.DataFrame(values[1:], columns=values[0])
    if "金額" in df.columns:
        df["金額"] = pd.to_numeric(df["金額"], errors="coerce").fillna(0.0)
    return df
# -----------------------------------------------------------------------

# --- AI 回饋函數 (已優化，針對不同分類給予建議) ---
def generate_daily_feedback(df: pd.DataFrame) -> str:
    """
    根據當日資料提供更具體的分類回饋。
    """
    if df.empty:
        return "🤖 歡迎使用！開始新增第一筆記錄吧。"

    # 1. 篩選出今日的資料
    today_str = datetime.date.today().strftime("%Y-%m-%d")
    df_today = df[df['日期'] == today_str].copy()

    if df_today.empty:
        return f"🤖 今天 ({today_str}) 還沒有新增支出或收入記錄喔！"

    # 確保金額是數字
    df_today['金額'] = pd.to_numeric(df_today['金額'], errors='coerce').fillna(0)
    df_expense = df_today[df_today['分類'] != '收入']

    # 2. 計算今日總額與淨流入
    total_spend_today = df_expense['金額'].sum()
    total_income_today = df_today[df_today['分類'] == '收入']['金額'].sum()

    feedback = [f"🤖 **今日財務小結 ({today_str}):**\n"]

    # 3. 規則：總體支出警告 (假設超過 $800 為多)
    SPEND_LIMIT = 800
    if total_spend_today > SPEND_LIMIT:
        feedback.append(f"🔴 **總支出警報！** 今日已花費 **${total_spend_today:.0f}**，已超過 ${SPEND_LIMIT} 的每日預算建議。")

    # 4. 規則：針對個別分類的建議
    if not df_expense.empty:

        # 按分類彙總今日支出
        daily_cat_summary = df_expense.groupby('分類')['金額'].sum().sort_values(ascending=False)

        for category, amount in daily_cat_summary.items():
            if amount == 0:
                continue

            # --- 餐飲建議 ---
            if category == '餐飲' and amount > 500:
                feedback.append(f"🟡 **餐飲 (${amount:.0f})：** 今天的餐飲花費似乎有點高喔。嘗試自備或找找更實惠的餐廳，或許能省下一筆！")
            elif category == '餐飲' and amount > 300:
                feedback.append(f"👍 **餐飲 (${amount:.0f})：** 支出合理。保持良好的飲食與消費習慣！")

            # --- 交通建議 ---
            elif category == '交通' and amount > 150:
                feedback.append(f"⚠️ **交通 (${amount:.0f})：** 今日交通費用較高。可以考慮多利用大眾運輸的月票或騎乘單車，以減少長期開銷。")
            elif category == '交通' and amount > 0:
                 feedback.append(f"🟢 **交通 (${amount:.0f})：** 交通規劃得不錯，這是通勤支出的合理範圍。")

            # --- 娛樂建議 ---
            elif category == '娛樂' and amount > 1000:
                feedback.append(f"🔥 **娛樂 (${amount:.0f})：** 娛樂是必要的，但今日娛樂消費偏高！請確保這筆支出在當月預算內，避免透支。")
            elif category == '娛樂' and amount > 0:
                feedback.append(f"✨ **娛樂 (${amount:.0f})：** 保持適度的放鬆很重要。找找免費或低成本的娛樂活動，讓樂趣與省錢並行！")

            # --- 其他/生活用品建議 ---
            elif category in ['生活用品','學習','其他'] and amount > 400:
                feedback.append(f"💡 **{category} (${amount:.0f})：** {category}有大筆支出。建議檢查是否有非必要性或可以延後的採買項目。")

    # 5. 規則：收入提醒 (放在最後總結)
    if total_income_today > 0:
         feedback.append(f"\n🟢 **收入 (${total_income_today:.0f})：** 今日有資金進帳，為您的財務狀況加分！")

    # 6. 規則：總結
    net_flow = total_income_today - total_spend_today
    if net_flow < 0 and total_spend_today <= SPEND_LIMIT:
        feedback.append(f"\n😊 **總體評價：** 支出在控制範圍內。總淨流出 ${abs(net_flow):.0f}。")
    elif net_flow >= 0 and total_spend_today <= SPEND_LIMIT:
        feedback.append(f"\n🎉 **總體評價：** 收支平衡良好！淨流入 ${net_flow:.0f}。")
    elif total_spend_today > SPEND_LIMIT and net_flow < 0:
        feedback.append(f"\n🧐 **總體評價：** 今日淨流出 ${abs(net_flow):.0f}。記得查看高額支出建議，以調整未來預算！")

    return "\n".join(feedback)
# -----------------------------------------------------------------------


# --- 圖表繪製與彙總函式 (維持 AA > 2 人的邏輯) ---
def _make_summary_tables_and_plot(df: pd.DataFrame):
    if "分類" in df.columns:
        by_cat = df.groupby("分類", as_index=False)["金額"].sum().sort_values("金額", ascending=False)
        by_cat_plot = by_cat[by_cat['分類'] != '收入']
        fig = px.pie(by_cat_plot, values='金額', names='分類', title='支出分類佔比')
        fig.update_traces(textposition='inside', textinfo='percent+label')
    else:
        by_cat = pd.DataFrame(columns=["分類", "金額"])
        fig = gr.Plot()

    if "付款人" in df.columns and df["付款人"].notna().any():
        payers = df["付款人"].replace("", "匿名")
        unique_payers = sorted([p for p in payers.unique() if p])

        # 檢查付款人數：兩位或更多才計算 AA
        if len(unique_payers) >= 2:
            total = df["金額"].sum()
            n = len(unique_payers)
            aa_share = total / n
            paid_by = df.groupby("付款人", as_index=False)["金額"].sum().rename(columns={"金額": "實付"})

            all_rows = pd.DataFrame({"付款人": unique_payers}).merge(paid_by, on="付款人", how="left").fillna({"實付": 0.0})
            all_rows["應付(AA)"] = aa_share
            all_rows["差額(實付-應付)"] = all_rows["實付"] - all_rows["應付(AA)"]
            settle = all_rows.sort_values("差額(實付-應付)", ascending=False)
        else:
            # 少於兩位付款人，回傳空 DataFrame
            settle = pd.DataFrame(columns=["付款人", "實付", "應付(AA)", "差額(實付-應付)"])
    else:
        # 沒有付款人資料，回傳空 DataFrame
        settle = pd.DataFrame(columns=["付款人", "實付", "應付(AA)", "差額(實付-應付)"])

    return by_cat, settle, fig
# -----------------------------------------------------------------------

# --- 核心邏輯函式 (新增 AI 回饋輸出) ---
def add_expense(date_str, time_str, category, item, amount, payer, location, pay_way, notes):
    try:
        # ... (輸入處理邏輯維持不變) ...
        if not date_str:
            date_str = datetime.date.today().strftime("%Y-%m-%d")

        datetime.datetime.strptime(date_str, "%Y-%m-%d")
        if time_str:
            datetime.datetime.strptime(time_str, "%H:%M")

        category = (category or "未填").strip()
        item = (item or "未填").strip()
        payer = (payer or "匿名").strip()
        location = (location or "").strip()
        pay_way = (pay_way or "未填").strip()
        notes = (notes or "").strip()

        try:
            amount_val = float(amount)
        except:
            return (f"金額格式錯誤，請輸入數字", None, None, None, None, "", # 錯誤時回傳空回饋
                    gr.update(value=date_str), gr.update(value=time_str), gr.update(value=category), gr.update(value=item), gr.update(value=amount), gr.update(value=payer), gr.update(value=location), gr.update(value=pay_way), gr.update(value=notes))

        ws = _ensure_auth_and_ws()
        ws.append_row(
            [date_str, time_str or "", category, item, amount_val, payer, location, pay_way, notes],
            value_input_option="USER_ENTERED"
        )
        msg = f"✅ 已新增：{date_str}｜{category}｜{item}｜{amount_val}｜{payer}..."

        df = _read_df()
        cat, settle, fig = _make_summary_tables_and_plot(df)
        total = float(df["金額"].sum()) if not df.empty else 0.0

        # 【新增：生成 AI 回饋】
        feedback = generate_daily_feedback(df)

        # 成功後：更新輸出 (新增 feedback)
        return (msg, total, cat, settle, fig, feedback,
                gr.update(value=datetime.date.today().strftime("%Y-%m-%d")),
                gr.update(value=datetime.datetime.now().strftime("%H:%M")),
                gr.update(value=None),
                gr.update(value=""),
                gr.update(value=None),
                gr.update(value=""),
                gr.update(value=""),
                gr.update(value='現金'),
                gr.update(value="")
        )
    except Exception as e:
        # 錯誤訊息回傳 (新增 feedback 輸出)
        return (f"❌ 新增失敗：{e}", None, None, None, None, "❌ 新增失敗，無法生成回饋。",
                gr.update(value=date_str), gr.update(value=time_str), gr.update(value=category), gr.update(value=item), gr.update(value=amount), gr.update(value=payer), gr.update(value=location), gr.update(value=pay_way), gr.update(value=notes))

def refresh_summary():
    try:
        df = _read_df()
        if df.empty:
            return "目前沒有資料", 0.0, pd.DataFrame(), pd.DataFrame(), gr.Plot(), pd.DataFrame(), generate_daily_feedback(df)
        total = float(df["金額"].sum())
        by_cat, settle, fig = _make_summary_tables_and_plot(df)

        # 【新增：生成 AI 回饋】
        feedback = generate_daily_feedback(df)

        return "✅ 已更新彙總", total, by_cat, settle, fig, df, feedback
    except Exception as e:
        return f"❌ 讀取失敗：{e}", 0.0, pd.DataFrame(), pd.DataFrame(), gr.Plot(), pd.DataFrame(), "❌ 讀取資料失敗，無法生成回饋。"
# -----------------------------------------------------------------------

with gr.Blocks(title="日常支出速算與分攤") as demo:
    gr.Markdown("## 🧾 日常支出速算與分攤（Gradio）\n- 新增支出後自動寫回 Google Sheet\n- 一鍵查看總額、分類小計與 AA 分攤\n- 讀寫工作表：`工作表1`")

    category_choices = ['餐飲', '交通', '娛樂', '生活用品', '學習', '收入', '其他']
    pay_way_choices = ['現金', '信用卡', '電子支付', '轉帳', '其他']

    with gr.Tab("➕ 新增支出"):
        gr.Markdown("### 填寫交易資訊（新增後自動清空）")

        with gr.Row():
            date_in = gr.Textbox(label="日期 YYYY-MM-DD", value=datetime.date.today().strftime("%Y-%m-%d"))
            time_in = gr.Textbox(label="時間 HH:MM", value=datetime.datetime.now().strftime("%H:%M"))

        with gr.Row():
            cat_in = gr.Dropdown(label="分類", choices=category_choices, value='餐飲')
            item_in = gr.Textbox(label="品項", placeholder="如 咖啡 / 便當 / 車票")

        with gr.Row():
            amt_in = gr.Number(label="金額", placeholder="數字")
            payer_in = gr.Textbox(label="付款人", placeholder="如 Pecu / Alice / Bob")

        with gr.Row():
            location_in = gr.Textbox(label="購買地點", placeholder="如 全家 / 餐廳名稱")
            pay_way_in = gr.Dropdown(label="支付方式", choices=pay_way_choices, value='現金')

        with gr.Row():
            notes_in = gr.Textbox(label="交易描述/備註", placeholder="如 朋友生日禮物、公司報銷", lines=2)

        add_btn = gr.Button("新增到工作表並清空表單", variant="primary")

        # 輸出元件 (包含輸入元件以供清空)
        add_msg = gr.Markdown()
        total_out = gr.Number(label="目前總額", interactive=False)
        cat_df = gr.Dataframe(label="分類小計", interactive=False)
        settle_df = gr.Dataframe(label="AA 分攤結算", interactive=False)
        pie_plot = gr.Plot(label="支出分類圓餅圖")

        # 【新增：AI 回饋元件】
        daily_feedback_out_add = gr.Markdown(label="今日 AI 智慧回饋")

        # 按鈕點擊事件：inputs 和 outputs 順序與 add_expense 參數一致
        add_btn.click(
            fn=add_expense,
            inputs=[date_in, time_in, cat_in, item_in, amt_in, payer_in, location_in, pay_way_in, notes_in],
            outputs=[
                add_msg, total_out, cat_df, settle_df, pie_plot, daily_feedback_out_add, # <-- 新增回饋輸出
                date_in, time_in, cat_in, item_in, amt_in, payer_in, location_in, pay_way_in, notes_in # 清空輸入
            ]
        )

    with gr.Tab("📊 彙總 / 視覺化"):
        refresh_btn = gr.Button("讀取最新彙總與圖表", variant="secondary")
        msg2 = gr.Markdown()

        with gr.Row():
            total2 = gr.Number(label="總額", interactive=False)

        with gr.Row():
            cat_plot = gr.Plot(label="支出分類圓餅圖")

        with gr.Row():
            cat_df2 = gr.Dataframe(label="分類小計", interactive=False)
            settle_df2 = gr.Dataframe(label="AA 分攤結算", interactive=False)

        raw_preview = gr.Dataframe(label="（預覽）最近資料", interactive=False)
        # 【新增：AI 回饋元件】
        daily_feedback_out_summary = gr.Markdown(label="今日 AI 智慧回饋")

        refresh_btn.click(
            fn=refresh_summary,
            inputs=[],
            outputs=[msg2, total2, cat_df2, settle_df2, cat_plot, raw_preview, daily_feedback_out_summary] # <-- 新增回饋輸出
        )

    with gr.Tab("📒 檢視原始資料"):
        view_btn = gr.Button("讀取全部資料", variant="secondary")
        view_df = gr.Dataframe(label="全部資料", interactive=False)

        def _view_all():
            try:
                df = _read_df()
                if df.empty:
                    return pd.DataFrame(columns=REQUIRED_COLUMNS)
                return df
            except Exception as e:
                return pd.DataFrame({"錯誤": [str(e)]})

        view_btn.click(fn=_view_all, inputs=[], outputs=[view_df])

# 啟動介面
demo.launch(share=True)